In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Use the print() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

81.87457226644696 12.40869829267595
-77.80726323331243 115.78025195812728
-40.204847532681754 128.48583327222684
-72.22823814783719 -134.00112716703762
-44.86027258460529 109.31602462913736
36.668815975902476 -115.79339786223451
8.272843230303181 78.48380294590169
76.26799594056115 -17.13555607299199
63.23791119314342 -75.25948514162535
-61.80180402020507 125.9272716614147
86.82801244773398 -32.952389672386516
0.5971789207104763 -173.50505919949612
20.664054959816454 114.18770972385988
-45.06678446101398 -170.51549270997643
64.41306803145866 -22.534656099283325
-14.404895724550471 -4.756032476219929
-61.19653984728262 -163.325867101683
-12.958093169842982 -126.4023551079731
-68.35879426724013 -58.96633176636254
79.86309342970807 -24.85360472066978
57.835193368129524 -119.25495836647875
-68.60833408196102 103.85879197868968
-38.96518376373822 -44.264544286551256
36.378415244281726 119.43639365229353
-8.544695745001647 -98.65076207100785
-35.21456311068518 -42.13699182704519
68.169175639

In [5]:
from citipy import citipy

In [6]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

613

In [7]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [12]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [13]:
# Create an endpoint URL for a city.
city_url = url + "&q=" + "Boston"
print(city_url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=3d059a32267006c2165ae4fa6146124d&q=Boston


In [15]:
# Make a 'Get' request for the city weather.
city_weather = requests.get(city_url)
city_weather.status_code

200

In [16]:
# Get the JSON text of the 'Get' request.
city_weather.json()

{'coord': {'lon': -71.0598, 'lat': 42.3584},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 57.69,
  'feels_like': 57.52,
  'temp_min': 54.63,
  'temp_max': 60.8,
  'pressure': 1008,
  'humidity': 93},
 'visibility': 10000,
 'wind': {'speed': 1.01, 'deg': 258, 'gust': 3},
 'clouds': {'all': 58},
 'dt': 1664254497,
 'sys': {'type': 2,
  'id': 2013408,
  'country': 'US',
  'sunrise': 1664275010,
  'sunset': 1664318012},
 'timezone': -14400,
 'id': 4930956,
 'name': 'Boston',
 'cod': 200}

In [17]:
# Get the JSON data.
boston_data = city_weather.json()

In [19]:
boston_data['sys']['country']

'US'

In [20]:
boston_data['dt']

1664254497

In [22]:
# Import the datetime module from the datetime library.
from datetime import datetime
# Get the date from the JSON file.
date = boston_data["dt"]
# Convert the UTC date to a date format with year, month, day, hours, minutes, and seconds.
datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')

'2022-09-27 04:54:57'

In [23]:
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [24]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 2 of Set 1 | albany
Processing Record 3 of Set 1 | esperance
Processing Record 4 of Set 1 | rikitea
Processing Record 5 of Set 1 | busselton
Processing Record 6 of Set 1 | pahrump
Processing Record 7 of Set 1 | tiruchchendur
Processing Record 8 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 9 of Set 1 | iqaluit
Processing Record 10 of Set 1 | saleaula
City not found. Skipping...
Processing Record 11 of Set 1 | zhuhai
Processing Record 12 of Set 1 | vaini
Processing Record 13 of Set 1 | keflavik
Processing Record 14 of Set 1 | jamestown
Processing Record 15 of Set 1 | avarua
Processing Record 16 of Set 1 | atuona
Processing Record 17 of Set 1 | ushuaia
Processing Record 18 of Set 1 | fairview
Processing Record 19 of Set 1 | rio grande
Processing Record 20 of Set 1 | anqiu
Processing Record 21 of Set 1 | puerto ayo

Processing Record 35 of Set 4 | karratha
Processing Record 36 of Set 4 | souillac
Processing Record 37 of Set 4 | lake charles
Processing Record 38 of Set 4 | aracaju
Processing Record 39 of Set 4 | kaitangata
Processing Record 40 of Set 4 | katsuura
Processing Record 41 of Set 4 | monticello
Processing Record 42 of Set 4 | mazagao
Processing Record 43 of Set 4 | ketchikan
Processing Record 44 of Set 4 | palmer
Processing Record 45 of Set 4 | ilulissat
Processing Record 46 of Set 4 | yunjinghong
City not found. Skipping...
Processing Record 47 of Set 4 | comodoro rivadavia
Processing Record 48 of Set 4 | mocuba
Processing Record 49 of Set 4 | portland
Processing Record 50 of Set 4 | vao
Processing Record 1 of Set 5 | mehamn
Processing Record 2 of Set 5 | vangaindrano
Processing Record 3 of Set 5 | fehergyarmat
Processing Record 4 of Set 5 | rosita
Processing Record 5 of Set 5 | divnomorskoye
Processing Record 6 of Set 5 | saint-francois
Processing Record 7 of Set 5 | sur
Processing Rec

Processing Record 22 of Set 8 | talnakh
Processing Record 23 of Set 8 | campbellton
Processing Record 24 of Set 8 | mittagong
Processing Record 25 of Set 8 | lesnikovo
Processing Record 26 of Set 8 | angamacutiro
City not found. Skipping...
Processing Record 27 of Set 8 | sfantu gheorghe
Processing Record 28 of Set 8 | kaabong
Processing Record 29 of Set 8 | necochea
Processing Record 30 of Set 8 | chokurdakh
Processing Record 31 of Set 8 | el campo
Processing Record 32 of Set 8 | saint-georges
Processing Record 33 of Set 8 | tarnogskiy gorodok
Processing Record 34 of Set 8 | nurota
Processing Record 35 of Set 8 | teguldet
Processing Record 36 of Set 8 | alexandria
Processing Record 37 of Set 8 | aiken
Processing Record 38 of Set 8 | bamnet narong
Processing Record 39 of Set 8 | broome
Processing Record 40 of Set 8 | san rafael
Processing Record 41 of Set 8 | dhidhdhoo
Processing Record 42 of Set 8 | buxar
Processing Record 43 of Set 8 | venado tuerto
Processing Record 44 of Set 8 | ra

Processing Record 7 of Set 12 | mackay
Processing Record 8 of Set 12 | poddorye
Processing Record 9 of Set 12 | whitehorse
Processing Record 10 of Set 12 | sumbe
Processing Record 11 of Set 12 | brae
Processing Record 12 of Set 12 | hasaki
Processing Record 13 of Set 12 | mareeba
Processing Record 14 of Set 12 | mosquera
Processing Record 15 of Set 12 | saint combs
City not found. Skipping...
Processing Record 16 of Set 12 | solnechnyy
Processing Record 17 of Set 12 | bannu
Processing Record 18 of Set 12 | ye
City not found. Skipping...
Processing Record 19 of Set 12 | ballina
Processing Record 20 of Set 12 | caravelas
Processing Record 21 of Set 12 | nikel
Processing Record 22 of Set 12 | arkhangelskoye
Processing Record 23 of Set 12 | nyurba
Processing Record 24 of Set 12 | aden
Processing Record 25 of Set 12 | banda aceh
Processing Record 26 of Set 12 | la palma
Processing Record 27 of Set 12 | menongue
Processing Record 28 of Set 12 | yantal
Processing Record 29 of Set 12 | inhamba

In [25]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,42.6001,-73.9662,53.73,88,10,4.59,US,2022-09-27 05:39:44
1,Esperance,-33.8667,121.9000,64.17,60,0,10.36,AU,2022-09-27 05:39:44
2,Rikitea,-23.1203,-134.9692,68.43,60,68,9.46,PF,2022-09-27 05:39:45
3,Busselton,-33.6500,115.3333,64.35,59,0,7.72,AU,2022-09-27 05:39:45
4,Pahrump,36.2083,-115.9839,77.25,15,0,4.59,US,2022-09-27 05:36:30
5,Tiruchchendur,8.4833,78.1167,93.49,52,6,6.08,IN,2022-09-27 05:39:45
6,Iqaluit,63.7506,-68.5145,40.73,80,100,15.01,CA,2022-09-27 05:36:42
7,Zhuhai,22.2769,113.5678,95.86,74,40,17.27,CN,2022-09-27 05:39:46
8,Vaini,-21.2000,-175.2000,75.36,88,75,10.36,TO,2022-09-27 05:39:46
9,Keflavik,64.0049,-22.5624,41.43,65,0,3.44,IS,2022-09-27 05:39:47


In [26]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Albany,US,2022-09-27 05:39:44,42.6001,-73.9662,53.73,88,10,4.59
1,Esperance,AU,2022-09-27 05:39:44,-33.8667,121.9000,64.17,60,0,10.36
2,Rikitea,PF,2022-09-27 05:39:45,-23.1203,-134.9692,68.43,60,68,9.46
3,Busselton,AU,2022-09-27 05:39:45,-33.6500,115.3333,64.35,59,0,7.72
4,Pahrump,US,2022-09-27 05:36:30,36.2083,-115.9839,77.25,15,0,4.59
5,Tiruchchendur,IN,2022-09-27 05:39:45,8.4833,78.1167,93.49,52,6,6.08
6,Iqaluit,CA,2022-09-27 05:36:42,63.7506,-68.5145,40.73,80,100,15.01
7,Zhuhai,CN,2022-09-27 05:39:46,22.2769,113.5678,95.86,74,40,17.27
8,Vaini,TO,2022-09-27 05:39:46,-21.2000,-175.2000,75.36,88,75,10.36
9,Keflavik,IS,2022-09-27 05:39:47,64.0049,-22.5624,41.43,65,0,3.44


In [28]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")